## Black-Scholes

In [2]:
import QuantLib as ql

In [3]:
valuationDate = ql.Date(17, 4, 2020)
ql.Settings.instance().evaluationDate = valuationDate # 평가기준일 설정
calendar = ql.SouthKorea()
dayCount = ql.ActualActual()

In [4]:
underlying_qt = ql.SimpleQuote(270.48) # 기초자산가격
dividend_qt = ql.SimpleQuote(0.0) # 배당률
riskfreerate_qt = ql.SimpleQuote(0.01) # 무위험금리
volatility_qt = ql.SimpleQuote(0.13) # 변동성

In [5]:
# Handel - Simple 값이 변해도 바로 받아들일수 있게,
u_qhd = ql.QuoteHandle(underlying_qt)
q_qhd = ql.QuoteHandle(dividend_qt)
r_qhd = ql.QuoteHandle(riskfreerate_qt)
v_qhd = ql.QuoteHandle(volatility_qt)

Term-Structure Objects

In [6]:
r_ts = ql.FlatForward(valuationDate, r_qhd, dayCount)
d_ts = ql.FlatForward(valuationDate, q_qhd, dayCount)
v_ts = ql.BlackConstantVol(valuationDate, calendar, v_qhd, dayCount)

# 만기에 따른 기간구조 형성하는 것 3가지

Term-Structure Handle Objects

In [8]:
r_thd = ql.YieldTermStructureHandle(r_ts)
d_thd = ql.YieldTermStructureHandle(d_ts)
v_thd = ql.BlackVolTermStructureHandle(v_ts)
# 앞과 같이 Handle에 넘겨주는 작업

Process & Engine

In [9]:
process = ql.BlackScholesMertonProcess(u_qhd, d_thd, r_thd, v_thd)
# 시장 데이터가 어떤 확률과정을 따를지
engine = ql.AnalyticEuropeanEngine(process)
# 어떤 방법 사용하는 엔진인지

Option Objects

In [16]:
option_type = ql.Option.Call
strikePrice = 272                                        #행사가
expiryDate = ql.Date(31,12,2021)
exercise = ql.EuropeanExercise(expiryDate)
payoff = ql.PlainVanillaPayoff(option_type, strikePrice)
option = ql.VanillaOption(payoff, exercise)

Pricing

In [18]:
option.setPricingEngine(engine)

Price & Greeks Results

In [19]:
print("Option Premium =", round(option.NPV(), 2))
print("Option Delta =", round(option.delta(), 4))
print("Option Gamma =", round(option.gamma(), 4))
print("Option Theta =", round(option.thetaPerDay(), 4))
print("Option Vega =", round(option.vega()/ 100, 4))
print("Option Rho =", round(option.rho()/ 100, 4))

Option Premium = 12.7
Option Delta = 0.6558
Option Gamma = 0.0173
Option Theta = -0.0109
Option Vega = 1.3217
Option Rho = 2.8584


Automatic Re-Pricing 주가가 변했을 때

In [20]:
underlying_qt.setValue(275) # 주가가 275로 변했을 때
print("Option Premium =", round(option.NPV(), 2))
print("Option Delta =", round(option.delta(), 4))
print("Option Gamma =", round(option.gamma(), 4))
print("Option Theta =", round(option.thetaPerDay(), 4))
print("Option Vega =", round(option.vega()/ 100, 4))
print("Option Rho =", round(option.rho()/ 100, 4))

Option Premium = 12.7
Option Delta = 0.6558
Option Gamma = 0.0173
Option Theta = -0.0109
Option Vega = 1.3217
Option Rho = 2.8584


Implied Volatility (내재변동성 구하기)

In [21]:
mkt_price = 8.21 # 콜옵션 가격
implied_volatility = option.impliedVolatility(mkt_price, process)
volatility_qt.setValue(implied_volatility) # 옵션의 시장가격과 확률과정을 받음
print("Option Premium =", round(option.NPV(), 2))
print("Option Delta =", round(option.delta(), 4))
print("Option Gamma =", round(option.gamma(), 4))
print("Option Theta =", round(option.thetaPerDay(), 4))
print("Option Vega =", round(option.vega()/ 100, 4))
print("Option Rho =", round(option.rho()/ 100, 4))

Option Premium = 8.21
Option Delta = 0.8474
Option Gamma = 0.031
Option Theta = -0.0076
Option Vega = 0.8467
Option Rho = 3.8333
